In [41]:
%matplotlib inline
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import Tensor,optim
import numpy as np
import scipy.io
import pickle
import h5py
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import tqdm
import itertools
from scipy.signal import butter,lfilter, filtfilt, freqz, find_peaks_cwt, medfilt, find_peaks, resample
from trial import Trial
from tqdm import tnrange
import os
import sys
import pandas as pd
import pingouin as pg
np.random.seed(sum(map(ord, "aesthetics")))

#os.getcwd()

print(f"Is CUDA supported by this system?{torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
  
# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device:{torch.cuda.current_device()}")    
print(f"Name of current CUDA device:{torch.cuda.get_device_name(cuda_id)}")
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

Is CUDA supported by this system?True
CUDA version: 11.3
ID of current CUDA device:0
Name of current CUDA device:NVIDIA GeForce GTX 745
ERROR! Session/line number was not unique incuda
 database. History logging moved to new session 157


In [42]:
def get_by_animal(trial_list, name):
    return list(filter(lambda trial: trial.name==name,trial_list))

def get_sessions(trial_list):
    sessions = list()
    for trial in trial_list:
        if not trial.session in sessions:
            sessions.append(trial.session)
    sessions.sort()
    return sessions

def get_by_session(trial_list, session):
    return list(filter(lambda trial: trial.session==session,trial_list))

def get_by_date(trial_list, date):
    date = str(date)
    return list(filter(lambda trial: trial.date==date,trial_list))

def get_by_session_type(trial_list, session_type):
    return list(filter(lambda trial: trial.session_type==session_type,trial_list))

def get_performance(trial_list):
    total = 0.0
    for trial in trial_list:
        total += int(trial.correct)
    return total/len(trial_list)

def get_correct_trials(trial_list):
    total = 0.0
    for trial in trial_list:
        if trial.correct == True:
            print(1)
            
        else:
            print(0)

def get_by_delay(trial_list, delay):
    return list(filter(lambda trial: trial.delay_time==delay,trial_list))

def get_by_contrast(trial_list, contrast):
    return list(filter(lambda trial: round(trial.high_count/trial.low_count)==contrast,trial_list))

def get_performance_list(trials):
    sessions = get_sessions(trials)
    performance_list = list()
    for session in sessions:
        performance_list.append(get_performance(get_by_session(trials,session)))
    return performance_list

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y

def get_inhalation(trial):
    breath = butter_lowpass_filter(trial.breath_profile, cutoff = 10.0, fs = 1000.0, order = 6)
    breath = breath - breath.mean()
    breath = breath / (breath.max() - breath.min()) * 2
    inhalation = np.gradient(breath) < -0.00
    return inhalation.astype(float)

def get_odor_profile_actual(trial):
    kernel = np.load('odor_kernel_50ms.npy')
    pulse_onsets = (np.diff(trial.odor_profile)>0).astype(float)
    pulse_onsets_shifted = np.concatenate((np.zeros((70,)),pulse_onsets,np.zeros(131)))
    odor_profile_actual = np.convolve(pulse_onsets_shifted,kernel,mode='same')
    return odor_profile_actual

def get_full_inhalation(trial):
    breath = butter_lowpass_filter(np.concatenate((trial.pre_breath_profile,trial.breath_profile)), 
                                   cutoff = 10.0, fs = 1000.0, order = 6)
    breath = breath - breath.mean()
    breath = breath / (breath.max() - breath.min()) * 2
    inhalation = np.gradient(breath) < -0.00
    return inhalation.astype(float)

def get_full_inhalation_test(trial):
    breath = butter_lowpass_filter(np.concatenate((trial.pre_breath_profile,trial.breath_profile)), 
                                   cutoff = 10.0, fs = 1000.0, order = 6)
    breath = breath - breath.mean()
    breath = breath / (breath.max() - breath.min()) * 2
    breath[breath>0] = 0
    return -breath
    

def get_inhaled_odor(trial):
    inhalation = get_inhalation(trial)
    odor_profile = get_odor_profile_actual(trial)
    total_odor = (inhalation[0:odor_profile.size]*odor_profile).sum()
    return total_odor

def get_inhaled_odor_trace(trial):
    sniff_mod_kernel = np.load('inhalation_kernel_fine_weights_active.npy')

    delay_time = trials[0].delay_time
    sniff = butter_lowpass_filter(trials[0].breath_profile,10,1000,3)[0:delay_time]
    sniff = (sniff - sniff.mean() +1)/2
    sniff_onset,_ = find_peaks(sniff,distance=150)
    num_sniffs = len(sniff_onset)
    odor_profile_actual = get_odor_profile_actual(trial)
    sniff_norm = np.zeros((num_sniffs,250))
    odor_norm = np.zeros((num_sniffs,250))

    sniff_marks = np.concatenate((sniff_onset[:-1].reshape(-1,1),sniff_onset[1:].reshape(-1,1)),axis=1)
    sniff_durations = np.diff(sniff_marks,axis=1)
    num_sniffs = sniff_marks.shape[0]
    sniff_mod = np.zeros(sniff.shape)
    for i in range(num_sniffs):
        sniff_mod[int(sniff_marks[i,0]):int(sniff_marks[i,1])] = resample(sniff_mod_kernel,int(sniff_durations[i]))
        
    inhaled_odor_trace =  odor_profile_actual[0:delay_time]*sniff_mod
    return inhaled_odor_trace

def get_by_pulses(trial_list, low_pulses, high_pulses):
    return list(filter(lambda trial: (trial.num_pulses>=low_pulses and trial.num_pulses<high_pulses),
                       trial_list))

def get_by_inhaled(trial_list, low, high):
    return list(filter(lambda trial: (get_inhaled_odor(trial)>=low and get_inhaled_odor(trial)<high),
                       trial_list))

In [43]:
#all_trials_BNPS = pickle.load(open('./Trials/all_trials_BNPS.pickle','rb'))
#all_trials_RCS = pickle.load(open("./Trials/all_trials_corrected.pickle",'rb'))
stark_exc = pickle.load(open('./Trials/Stark_20220223_3.pickle','rb'))
#n_exc = pickle.load(open('./Trials/Natasha_004.pickle','rb'))

In [44]:
#merged_dataset = all_trials_BNPS + all_trials_RCS
#random_trials_all = get_by_session_type(merged_dataset, "random")
#block_trials_all = get_by_session_type(merged_dataset, "block")

In [46]:
get_performance(get_by_date(stark_exc, 20220223))

0.9861111111111112

In [5]:
class Model(nn.Module):
    
    def __init__(self):
        
        super(Model, self).__init__()
        
    def build_model(self, input_dim, output_dim):
        model = torch.nn.Sequential().to(device)
        # computes w_c^T x + b_c 
        model.add_module("linear1", torch.nn.Linear(input_dim, output_dim,bias=True))
        # Compute our log softmax term.
        model.add_module("softmax", torch.nn.LogSoftmax(dim=1)) #We used softmax because is a 
        #generalized version of the sigmoid
        return model

    def train(self, model, loss, optimizer, x_val, y_val):

        # Take in x and y and make variable.
        x = Variable(x_val).to(device)
        y = Variable(y_val).to(device)
        
        model.to(device)
        # Resets the gradients to 0
        optimizer.zero_grad()

        # Computes the function above. (log softmax w_c^T x + b_c)
        fx = model.forward(x)

        # Computes a loss. Gives a scalar. 
        output = loss.forward(fx, y)

        # Magically computes the gradients. 
        output.backward()

        # updates the weights
        optimizer.step()
        return output.item()

    def predict(self, model, x_val):
        model.to(device)
        x = Variable(x_val, requires_grad=False)
        output = model.forward(x)
        return output.data.argmax(axis=1)   

In [6]:
class AutoLogReg(Model):
    
    def __init__(self, data, animals, contrasts, delay, trainsize, odor_k=True, inh_k=False, full_inh=False):
        
        super(Model).__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.data = data
        self.animals = animals
        self.contrasts = contrasts
        self.delay = delay
        self.trainsize = trainsize
        self._use_odor_kernel = odor_k
        self._use_inh_kernel = inh_k
        self._full_inhalation = full_inh
        self.nseg = None
        self.nsamples = None
        self.x_array = None
        self.z_array = None
        self.y_array = None
        self.x_raw = None
        
    def main(self, target,x_test,z_test,x_train,z_train):

        torch.manual_seed(42)  
        mod=Model()
        mod.to(self.device)
        #print(self.nsamples, self.nseg)
        n_examples, n_features = self.nsamples, self.nseg
        n_classes = 2
        model = mod.build_model(n_features, n_classes)

        # Loss here is negative log-likelihood 
        loss = torch.nn.NLLLoss()

        # Optimizer. SGD stochastic gradient. 
        optimizer = optim.Adam(model.parameters(), lr=0.005)

        batch_size = self.nsamples
        acc = 0
        i = 0
        
        while(i<1000):
            i += 1
            cost = 0.
            cost += mod.train(model, loss, optimizer,
                              x_train, z_train)
            predz = mod.predict(model, x_train)
            predz_test = mod.predict(model,x_test)
            acc = (100.0*(predz == z_train)).mean()
            acc_test = (100.0*(predz_test == z_test)).mean()
            if i%10 == 0:
                #print('epoch =',i,',cost =  %.2f' % cost, 'train performance = %.2f%%'%acc,'test performance = %.2f%%' %acc_test)
                pass
        return model, acc, acc_test, cost

    def validate(self, n,target): # x_array (odor profile passed through the kernel) 
        #prediction on z_array (licking decision)

        models = list()
        avg_acc = 0
        avg_acc_test = 0
        avg_cost = 0
        for i in tqdm.notebook.trange(n):
            ind_array = np.random.permutation(self.nsamples)
            x_shuffled = self.x_array[ind_array,:]
            z_shuffled = self.z_array[ind_array].squeeze()
            x_all = Tensor(x_shuffled.astype(float)).to(self.device)
            z_all = Tensor(z_shuffled.astype(float)).to(self.device)
            z_all = z_all.long().to(self.device)
            x_train = x_all[0:int(self.nsamples*self.trainsize),:]
            z_train = z_all[0:int(self.nsamples*self.trainsize)]
            x_test = x_all[int(self.nsamples*self.trainsize):,:]
            z_test = z_all[int(self.nsamples*self.trainsize):]
            model, acc, acc_test,cost = self.main(target,x_test,z_test,x_train,z_train)
            model.to(device)
            avg_acc += acc
            avg_acc_test += acc_test
            avg_cost += cost
            models.append(model)

        avg_acc/=n
        avg_acc_test/=n
        avg_cost/=n

        print("")
        print('validation rounds =',i+1,'train performance = %.2f%%'%avg_acc,'test performance = %.2f%%' %avg_acc_test,'cost = %.2f' %avg_cost)
        return models

    def validate_truth(self,n,target): # x_array (odor profile passed through the kernel) 
        #prediction on y_array (reward side)
        models = list()
        avg_acc = 0
        avg_acc_test = 0
        for i in tqdm.notebook.trange(n):
            ind_array = np.random.permutation(self.nsamples)
            x_shuffled = self.x_array[ind_array,:]
            z_shuffled = self.y_array[ind_array].squeeze()
            x_all = Tensor(x_shuffled.astype(float)).to(self.device)
            z_all = Tensor(z_shuffled.astype(float)).to(self.device)
            z_all = z_all.long()
            x_train = x_all[0:int(self.nsamples*self.trainsize),:]
            z_train = z_all[0:int(self.nsamples*self.trainsize)]
            x_test = x_all[int(self.nsamples*self.trainsize):,:]
            z_test = z_all[int(self.nsamples*self.trainsize):]
            model, acc, acc_test,cost = self.main(target,x_test,z_test,x_train,z_train)
            model.to(device)
            avg_acc += acc
            avg_acc_test += acc_test
            models.append(model)
        avg_acc/=n
        avg_acc_test/=n

        print("")
        print('validation rounds =',i+1,'train performance = %.2f%%'%avg_acc,'test performance = %.2f%%' %avg_acc_test)
        return models

    def validate_shuffled(self,n,target): #x_array (odor profile passed through the kernel) in which the time
        # course of the odor profile has been shuffled for each trial. The prediction is again on
        #z_array (licking decision, unshuffled)

        models = list()
        avg_acc = 0
        avg_acc_test = 0

        for i in tqdm.notebook.trange(n):
            x_raw_shuffled = np.zeros(self.x_raw.shape)
            for j in range(self.nsamples):
                ind_array = np.random.permutation(self.delay)
                x_raw_shuffled[j,:] = self.x_raw[j,ind_array]

            x_shuffled2 = x_raw_shuffled.reshape((self.nsamples,self.nseg,int(self.delay/self.nseg))).mean(axis=2).squeeze()
            ind_array = np.random.permutation(self.nsamples)
            x_shuffled = x_shuffled2[ind_array,:]
            z_shuffled = self.z_array[ind_array].squeeze()
            x_all = Tensor(x_shuffled.astype(float)).to(self.device)
            z_all = Tensor(z_shuffled.astype(float)).to(self.device)
            z_all = z_all.long()
            x_train = x_all[0:int(self.nsamples*self.trainsize),:]
            z_train = z_all[0:int(self.nsamples*self.trainsize)]
            x_test = x_all[int(self.nsamples*self.trainsize):,:]
            z_test = z_all[int(self.nsamples*self.trainsize):]
            model, acc, acc_test,cost = self.main(target,x_test,z_test,x_train,z_train)
            model.to(device)
            avg_acc += acc
            avg_acc_test += acc_test
            models.append(model)
        avg_acc/=n
        avg_acc_test/=n

        print("")
        print('validation rounds =',i+1,'train performance = %.2f%%'%avg_acc,'test performance = %.2f%%' %avg_acc_test)
        return models
    
    def autologreg(self, nrounds, binsize):
        
        self.nseg = self.delay//binsize
        #print(nseg)
        all_validated_weights = np.nan*np.ones((len(self.contrasts),len(self.animals),nrounds,self.nseg))
        all_shuffled_weights = np.nan*np.ones((len(self.contrasts),len(self.animals),nrounds,self.nseg))
        all_reward_weights = np.nan*np.ones((len(self.contrasts),len(self.animals),nrounds,self.nseg))

        td = get_by_delay(self.data,self.delay)

        for c in range(len(self.contrasts)):

            print('the contrast is:', self.contrasts[c])

            trialx = get_by_contrast(td,self.contrasts[c])

            for animal, animal_index in zip(self.animals, range(all_validated_weights.shape[1])): #Selection of the data for running the logistic regression analysis

                print(animal)
                trials = get_by_animal(trialx, animal)
    
                self.nsamples = len(trials)

                if self.nsamples != 0:
                    print(self.nsamples)
                    dt=0
                    pre_time = trials[0].refract_time

                    self.x_raw = np.zeros((self.nsamples,self.delay)) # Each row is a trial, each cell is a 1.25ms timepoint
                    self.x_array = np.zeros((self.nsamples,self.nseg)) # Each row is a trial, each cell is a 1 second timepoint
                    self.y_array = np.zeros((self.nsamples,)) # Each cell is a trial
                    self.z_array = np.zeros((self.nsamples,)) # Each cell is a trial

                    for i in range(self.nsamples): #looping over the lenght of the trial list
                        
                        self.y_array[i] = trials[i].reward_side #Stores the value of the correct side for each trial
                        self.z_array[i] = trials[i].lick_side #Stores the value of the mice decision for each trial
                        
                        if self._use_odor_kernel:
                            self.x_raw[i,:] = get_odor_profile_actual(trials[i])[0:self.delay] #allocates odor pulse 
                                                #profile for each trial into x_raw
                            #print('we will be making predictions based on data that went through odor kernel')
                                
                        elif self._use_inh_kernel:
                            self.x_raw[i,:] = get_inhaled_odor_trace(trials[i])
                            #print('we will be making predictions based on data that went through inhalation kernel')

                        elif self._full_inhalation:
                            self.x_raw[i,:] = get_full_inhalation(trials[i])[(pre_time+dt):(pre_time+dt+delay_time)] * get_odor_profile_actual(trials[i])[0:delay_time]
                            #print('we will be making predictions based on data that went through get_full_inhalation method')

                        
                    self.x_array = self.x_raw.reshape((self.nsamples,self.nseg,int(self.delay/self.nseg))).mean(axis=2).squeeze() #we take the mean for parameter reduction

                    ###############################################################################################
                    # Now I will run the logistic regression analysis to see how good pulse number predicts licking
                    ###############################################################################################

                    models = self.validate(nrounds,80) #Where this 80 comes from (target)
                    validated_weights = torch.zeros((nrounds, self.nseg))

                    for i,model in enumerate(models):
                        validated_weights[i,:] = model.linear1.weight[1].detach()

                    all_validated_weights[c,animal_index,:,:] = validated_weights

                    mean_validated_weights = validated_weights.mean(axis=0)
                    std_validated_weights = validated_weights.std(axis=0)

                    ###############################################################################################
                    # Same as above but with the temporal profile of odor pulses shuffled for each trial
                    ###############################################################################################

                    shuffled_models = self.validate_shuffled(nrounds,80)
                    weights_shuffled = torch.zeros((nrounds, self.nseg))

                    for i,model in enumerate(shuffled_models):
                        weights_shuffled[i,:] = model.linear1.weight[1].detach()

                    mean_weights_shuffled = weights_shuffled.mean(axis=0)
                    std_weights_shuffled = weights_shuffled.mean(axis=0)

                    all_shuffled_weights[c,animal_index,:,:] = weights_shuffled

                    ###############################################################################################
                    # Logistic regression using pulses to predict reward locations (~ideal observer)
                    ###############################################################################################    

                    reward_models = self.validate_truth(nrounds, 80)
                    weights_reward = torch.zeros((nrounds, self.nseg))

                    for i,model in enumerate(reward_models):
                        weights_reward[i,:] = model.linear1.weight[1].detach()

                    mean_weights_reward = weights_shuffled.mean(axis=0)
                    std_weights_reward = weights_shuffled.mean(axis=0)

                    all_reward_weights[c,animal_index,:,:] = weights_reward

                else:
                    pass

        results = np.stack((all_validated_weights, all_shuffled_weights, all_reward_weights))
        return results

In [7]:
def DFcompiler(arr1, delay):

    df_list = ['Validated', 'Shuffled', 'Truth']
    df_dict = {}
    for i in df_list:
        df_dict[i] = pd.DataFrame()

    for i, j in zip((range(len(arr1))), df_list):

        df = pd.DataFrame(data=[(*axes, v.item()) for axes, v in zip(itertools.product(*[range(i) for i in arr1.shape]), np.nditer(arr1))], columns=tuple('xyzvwu'))
    
    for k, types in zip((range(len(df['x'].unique()))), df_list):
        df=df.replace({'x':{k:types}})
    
    #con = range(len(df['y'].unique())).astype(int)
    
    for c, contr in zip((range(len(df['y'].unique()))), ctst):
        df=df.replace({'y':{c:contr}})
        
    for a, anim in zip((range(len(df['z'].unique()))), animales):
        df=df.replace({'z':{a:anim}})

    df=df.rename(columns={'x':'Dtype','t':'Delay','y':'Contrast','z':'Animal','v':'Repeat','w':'Bin', 'u': "Weight"})
    df.insert(1,'Delay',delay)

    return df

In [15]:
animales = ['Ragdoll', 'Banner', 'Natasha', 'Siamese', 'Calico', 'Peterparker', 'Stark']
contrastes = [2,3,4,6]
delays = [1250, 2500, 5000, 10000]
ctst = ['1:2','1:3','1:4','1:6']
dts = ['d1250', 'd2500', 'd5000', 'd10000']
df_dicts_90 = {}

for i in dts:
    df_dicts[i] = pd.DataFrame()
    
bodies_90 = [AutoLogReg(random_trials_all, animales, contrastes, delays[i], 0.9, 
                     odor_k=True, inh_k=False, full_inh=False) for i in range(len(delays))]

for i, q in zip((range(len(bodies))), dts):
    arrax = bodies_90[i].autologreg(100,625)
    df_dicts_90[q] = DFcompiler(arrax, bodies_90[i].delay)
    
final_df_90 = pd.concat(df_dicts_90, ignore_index = True)

the contrast is: 2
Ragdoll
Banner
736


  0%|          | 0/100 [00:00<?, ?it/s]


validation rounds = 100 train performance = 54.17% test performance = 53.36% cost = 0.69


  0%|          | 0/100 [00:00<?, ?it/s]


validation rounds = 100 train performance = 54.17% test performance = 53.91%


...


validation rounds = 100 train performance = 55.01% test performance = 46.24% cost = 0.68


  0%|          | 0/100 [00:00<?, ?it/s]


validation rounds = 100 train performance = 55.11% test performance = 47.21%


  0%|          | 0/100 [00:00<?, ?it/s]


validation rounds = 100 train performance = 100.00% test performance = 100.00%


In [19]:
final_df_80 = final_df

from pathlib import Path  
filepath = Path('./LR_with_odor_kernel/max_90_10.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
final_df_90.to_csv(filepath)

In [20]:
final_df_90

,Dtype,Delay,Contrast,Animal,Repeat,Bin,Weight
0,Validated,1250,1:2,Ragdoll,0,0,NaN
1,Validated,1250,1:2,Ragdoll,0,1,NaN
2,Validated,1250,1:2,Ragdoll,1,0,NaN
3,Validated,1250,1:2,Ragdoll,1,1,NaN
4,Validated,1250,1:2,Ragdoll,2,0,NaN
...,...,...,...,...,...,...,...
251995,Truth,10000,1:6,Stark,99,11,3.192791
251996,Truth,10000,1:6,Stark,99,12,3.397163
251997,Truth,10000,1:6,Stark,99,13,3.081673
251998,Truth,10000,1:6,Stark,99,14,3.583179


In [21]:
sns.set_style('whitegrid')
sns.set_context('poster')
all_fig_size = (15,5)

final_df_90.set_index('Bin', inplace=True)
final_df_90.groupby('Delay','Dtype','Animal')['Weights'].plot(legend=True)

ValueError: No axis named Dtype for object type DataFrame

In [13]:
for i in range(all_validated_weights.shape[0]):
    print(animals[i])
    
    plt.plot(np.linspace(0,delay_time/1000,nsegs),all_validated_weights[i].mean(axis=0)/sum(all_validated_weights[i].mean(axis=0))) # /mean_weights.sum()
    plt.plot(np.linspace(0,delay_time/1000,nsegs),all_shuffled_weights[i].mean(axis=0)/sum(all_shuffled_weights[i].mean(axis=0)))
    plt.plot(np.linspace(0,delay_time/1000,nsegs),all_reward_weights[i].mean(axis=0)/sum(all_reward_weights[i].mean(axis=0)))

    plt.xticks(np.linspace(0,delay_time/1000,5))
    plt.ylim(-0.01,0.3)
    plt.show()

NameError: name 'all_validated_weights' is not defined

In [ ]:
sns.set_style('white')
plt.figure(figsize=(6,4))
plt.plot(np.linspace(0,delay_time/1000,nsegs),mean_weights_truth/mean_weights_truth.sum(),'--',color='black')
plt.plot(np.linspace(0,delay_time/1000,nsegs),mean_weights_shuffled/mean_weights_shuffled.sum(),color='grey')
plt.plot(np.linspace(0,delay_time/1000,nsegs),mean_weights/mean_weights.sum(),color='blue')
#plt.plot(np.linspace(0,delay_time/1000,nsegs),np.zeros(nsegs),color='black')
#plt.legend(['truth','shuffled','actual'], loc="best")

plt.legend(['truth','shuffled','actual'])
plt.ylim(0,0.25)
plt.xticks(np.arange(0,10))
plt.xlabel('Time (s)')
#plt.title('3.5s 4:1')
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
#plt.savefig('1to3_2500_ms_logreg_all.svg', bbox_inches='tight')
plt.show()

In [ ]:
#with pd.option_context('display.max_rows', None,
 #                      'display.max_columns', None,
  #                     'display.precision', 3,
                     #  ):